In [47]:
import pandas as pd
from pymongo import MongoClient
import json
import requests

In [48]:
#conecta na API zeeng e extrai dados de clientes para cadastro
secret_chave='035a5784d9368a7edeead6f178a16fb8' #ideal criar um modelo de anonimização de chave
link_api='http://api.zeeng.com.br/empresa/getempresas?chave={}'.format(secret_chave)

resp = requests.get(link_api)
df_z= pd.DataFrame.from_dict(resp.json())

In [49]:
def remove_dados (Midia, Data_inicio,Data_fim):
    qry="db.{}.remove({{'Data': {{'$gt': ISODate('{}'),'$lt': ISODate('{}')}}}})".format(Midia,Data_inicio,Data_fim)
    #qry="db.{}.remove({{'Data': ".format(Midia)
    db.command(qry)

#função para adicionar coluna de id da zeeng nos dados externos
def insere_colunas(vdf):
    vdf = pd.merge(vdf,df_z, left_on='Marca',right_on='NomeEmpresa')
    vdf.drop(['NomeEmpresa','DataCadastro','Setor'], inplace=True, axis=1)
    return vdf

In [50]:
# Conecta no MongoDB
client = MongoClient('127.0.0.1')
db=client.admin
# Teste de conexão apenas
#serverStatusResult=db.command("serverStatus")
#print(serverStatusResult)

In [51]:
db=client.Midias

In [52]:
df= pd.read_excel('/home/rluvian/Downloads/base_midias.xlsx',sheet_name='TV')
# TV, FOLHETOS, IMPRESSOa

In [53]:
#Adiciona o id do sistema zeeng aos dados Cibracom
df = insere_colunas(df)

In [54]:
#Replica as linhas separadas por virgula
df=df.assign(Praça=df['Praça'].str.split(',')).explode('Praça')

In [55]:
#Caso seja um reprocessamento deleta os dados existentes no banco
db.TV.delete_many({'Data': {'$gte': min(df['Data']), '$lte': max(df['Data'])}})

In [56]:
#Insere os registros no banco
dic = df.to_dict(orient='row')
rs = db.TV.insert_many(dic)

/home/rluvian/.local/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [58]:
df= pd.read_excel('/home/rluvian/Downloads/base_midias.xlsx',sheet_name='FOLHETOS')
#Adiciona o id do sistema zeeng aos dados Cibracom
df = insere_colunas(df)
#Replica as linhas separadas por virgula
#df=df.assign(Praça=df['Praça'].str.split(',')).explode('Praça')
#Caso seja um reprocessamento deleta os dados existentes no banco
db.Folhetos.delete_many({'Data': {'$gte': min(df['Data']), '$lte': max(df['Data'])}})
#Insere os registros no banco
dic = df.to_dict(orient='row')
rs = db.Folhetos.insert_many(dic)

In [59]:
df= pd.read_excel('/home/rluvian/Downloads/base_midias.xlsx',sheet_name='IMPRESSA')
#Adiciona o id do sistema zeeng aos dados Cibracom
df = insere_colunas(df)
#Replica as linhas separadas por virgula
df=df.assign(Praça=df['Praça'].str.split(',')).explode('Praça')
#Caso seja um reprocessamento deleta os dados existentes no banco
db.Impressos.delete_many({'Data': {'$gte': min(df['Data']), '$lte': max(df['Data'])}})
#Insere os registros no banco
dic = df.to_dict(orient='row')
rs = db.Impressos.insert_many(dic)